In [1]:
from google.colab import files
files.upload()


{}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d kushagrapandya/visdrone-dataset


Dataset URL: https://www.kaggle.com/datasets/kushagrapandya/visdrone-dataset
License(s): GNU Affero General Public License 3.0
 99% 2.08G/2.10G [00:33<00:00, 117MB/s] 
100% 2.10G/2.10G [00:33<00:00, 67.2MB/s]


In [4]:
!unzip visdrone-dataset.zip -d visdrone_dataset


Streaming output truncated to the last 5000 lines.
  inflating: visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000235.jpg  
  inflating: visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000236.jpg  
  inflating: visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000237.jpg  
  inflating: visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000238.jpg  
  inflating: visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000239.jpg  
  inflating: visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000240.jpg  
  inflating: visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000241.jpg  
  inflating: visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000242.jpg  
  inflating: visdrone_dataset/VisDrone2019-DE

In [5]:
#Function to Change Annotations to YOLO
import os
import glob
import cv2

def visdrone_to_yolo(input_dir, output_dir, img_dir):
    """
    Converts VisDrone annotations to YOLOv8 format.

    Args:
        input_dir (str): Path to the directory containing VisDrone annotation files (.txt).
        output_dir (str): Path to save converted YOLOv8 annotation files.
        img_dir (str): Path to the directory containing the corresponding images.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Get all annotation files
    annotation_files = glob.glob(os.path.join(input_dir, "*.txt"))

    for file in annotation_files:
        # Get corresponding image file to determine resolution
        img_filename = os.path.splitext(os.path.basename(file))[0] + ".jpg"
        img_path = os.path.join(img_dir, img_filename)

        if not os.path.exists(img_path):
            print(f"Warning: Image {img_path} not found. Skipping.")
            continue

        img = cv2.imread(img_path)
        img_height, img_width, _ = img.shape

        with open(file, 'r') as f:
            lines = f.readlines()

        yolo_annotations = []
        for line in lines:
            elements = line.strip().split(',')
            if len(elements) < 8:
                continue  # Skip invalid lines

            bbox_left = int(elements[0])
            bbox_top = int(elements[1])
            bbox_width = int(elements[2])
            bbox_height = int(elements[3])
            class_id = int(elements[5])  # Object category

            # Convert to YOLO format
            center_x = (bbox_left + bbox_width / 2) / img_width
            center_y = (bbox_top + bbox_height / 2) / img_height
            norm_width = bbox_width / img_width
            norm_height = bbox_height / img_height

            yolo_annotations.append(f"{class_id} {center_x:.6f} {center_y:.6f} {norm_width:.6f} {norm_height:.6f}")

        # Save to output directory with the same filename
        output_file = os.path.join(output_dir, os.path.basename(file))
        with open(output_file, 'w') as f:
            f.write("\n".join(yolo_annotations))

    print(f"Converted {len(annotation_files)} files from VisDrone to YOLO format.")


In [6]:
#Create New Directory:
import os

# Define the main directory
base_dir = "Whole_Dataset"

# Define subdirectories
sub_dirs = ["images", "labels"]

# Create directories
os.makedirs(base_dir, exist_ok=True)
for sub_dir in sub_dirs:
    os.makedirs(os.path.join(base_dir, sub_dir), exist_ok=True)

print("Folder structure created successfully!")


Folder structure created successfully!


In [7]:
visdrone_to_yolo(
    input_dir="/content/visdrone_dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/annotations",
    output_dir="/content/Whole_Dataset/labels",
    img_dir="/content/visdrone_dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/images"
)


Converted 1610 files from VisDrone to YOLO format.


In [8]:
# Convert train annotations
visdrone_to_yolo(
    input_dir="/content/visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/annotations",
    output_dir="/content/Whole_Dataset/labels",
    img_dir="/content/visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images"
)

# Convert val annotations
visdrone_to_yolo(
    input_dir="/content/visdrone_dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/annotations",
    output_dir="/content/Whole_Dataset/labels",
    img_dir="/content/visdrone_dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/images"
)


Converted 6471 files from VisDrone to YOLO format.
Converted 548 files from VisDrone to YOLO format.


In [9]:
#Copy Images from Every where to Temp
import os
import shutil
from tqdm import tqdm

def copy_images(src_dirs, dest_dir):
    """
    Copies all images from multiple source directories to a single destination directory.

    Args:
        src_dirs (list): List of source directories containing images.
        dest_dir (str): Destination directory where images will be copied.
    """
    os.makedirs(dest_dir, exist_ok=True)  # Ensure destination folder exists

    for src_dir in src_dirs:
        if not os.path.exists(src_dir):
            print(f"Warning: Source directory {src_dir} does not exist. Skipping.")
            continue

        image_files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

        for img_file in tqdm(image_files, desc=f"Copying from {src_dir}"):
            src_path = os.path.join(src_dir, img_file)
            dest_path = os.path.join(dest_dir, img_file)

            # Ensure no overwriting by renaming duplicates
            counter = 1
            while os.path.exists(dest_path):
                filename, ext = os.path.splitext(img_file)
                new_filename = f"{filename}_{counter}{ext}"
                dest_path = os.path.join(dest_dir, new_filename)
                counter += 1

            shutil.copy2(src_path, dest_path)

    print("✅ All images copied successfully.")

# Define source directories and destination
source_dirs = [
    "/content/visdrone_dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/images",
    "/content/visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images",
    "/content/visdrone_dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/images"
]
destination_dir = "/content/Whole_Dataset/images"

# Run the function
copy_images(source_dirs, destination_dir)


Copying from /content/visdrone_dataset/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/images: 100%|██████████| 1610/1610 [00:00<00:00, 1831.59it/s]
Copying from /content/visdrone_dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images: 100%|██████████| 6471/6471 [00:07<00:00, 863.10it/s] 
Copying from /content/visdrone_dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/images: 100%|██████████| 548/548 [00:00<00:00, 2102.14it/s]

✅ All images copied successfully.


In [10]:
#Make the structure better
import os

old_path = "/content/Whole_Dataset"
new_path = "/content/Original_Annotation"

if os.path.exists(old_path):
    os.rename(old_path, new_path)
    print(f"✅ Renamed '{old_path}' to '{new_path}'")
else:
    print(f"❌ Folder '{old_path}' does not exist!")


✅ Renamed '/content/Whole_Dataset' to '/content/Original_Annotation'


In [11]:
#Make the Structure better :
import os

# Define the new directory structure
base_dir = "/content/Unified_Annotation"
sub_dirs = [
    "images/train",
    "images/val",
    "images/test",
    "labels/train",
    "labels/val",
    "labels/test"
]

# Create directories
for sub_dir in sub_dirs:
    os.makedirs(os.path.join(base_dir, sub_dir), exist_ok=True)

print(f"✅ Created '{base_dir}' with train, val, and test subfolders in images and labels.")


✅ Created '/content/Unified_Annotation' with train, val, and test subfolders in images and labels.


In [12]:
import os

# Rename Original_Annotation back to Whole_Data
old_path = "/content/Original_Annotation"
new_path = "/content/Whole_Data"

if os.path.exists(old_path):
    os.rename(old_path, new_path)
    print(f"✅ Renamed '{old_path}' to '{new_path}'")
else:
    print(f"❌ Folder '{old_path}' does not exist!")

# Create Original_Annotation with the same structure as Unified_Annotation
base_dir = "/content/Original_Annotation"
sub_dirs = [
    "images/train",
    "images/val",
    "images/test",
    "labels/train",
    "labels/val",
    "labels/test"
]

for sub_dir in sub_dirs:
    os.makedirs(os.path.join(base_dir, sub_dir), exist_ok=True)

print(f"✅ Created '{base_dir}' with train, val, and test subfolders in images and labels.")


✅ Renamed '/content/Original_Annotation' to '/content/Whole_Data'
✅ Created '/content/Original_Annotation' with train, val, and test subfolders in images and labels.


In [13]:
import os

# Define the directories for images and labels
image_dir = "/content/Whole_Data/images"
label_dir = "/content/Whole_Data/labels"

# Get list of image files
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

# Sort files to ensure consistent renaming order (optional)
image_files.sort()

# Initialize counter
counter = 0

# Rename images and corresponding annotation files
for img_file in image_files:
    # Construct the new name for the image and label
    new_img_name = f"{counter:05d}{os.path.splitext(img_file)[1]}"
    new_label_name = f"{counter:05d}.txt"

    # Define old and new paths for the image and label
    old_img_path = os.path.join(image_dir, img_file)
    new_img_path = os.path.join(image_dir, new_img_name)

    old_label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + ".txt")
    new_label_path = os.path.join(label_dir, new_label_name)

    # Rename the image and label
    if os.path.exists(old_label_path):
        os.rename(old_img_path, new_img_path)
        os.rename(old_label_path, new_label_path)
        print(f"Renamed {img_file} and corresponding label to {new_img_name} and {new_label_name}")
    else:
        print(f"Warning: Label for {img_file} not found. Skipping.")

    # Increment counter
    counter += 1

print("✅ Renaming completed.")


Streaming output truncated to the last 5000 lines.
Renamed 9999950_00000_d_0000006.jpg and corresponding label to 03630.jpg and 03630.txt
Renamed 9999950_00000_d_0000007.jpg and corresponding label to 03631.jpg and 03631.txt
Renamed 9999950_00000_d_0000008.jpg and corresponding label to 03632.jpg and 03632.txt
Renamed 9999950_00000_d_0000009.jpg and corresponding label to 03633.jpg and 03633.txt
Renamed 9999950_00000_d_0000010.jpg and corresponding label to 03634.jpg and 03634.txt
Renamed 9999950_00000_d_0000011.jpg and corresponding label to 03635.jpg and 03635.txt
Renamed 9999950_00000_d_0000012.jpg and corresponding label to 03636.jpg and 03636.txt
Renamed 9999950_00000_d_0000013.jpg and corresponding label to 03637.jpg and 03637.txt
Renamed 9999950_00000_d_0000014.jpg and corresponding label to 03638.jpg and 03638.txt
Renamed 9999950_00000_d_0000015.jpg and corresponding label to 03639.jpg and 03639.txt
Renamed 9999950_00000_d_0000016.jpg and corresponding label to 03640.jpg and 03

In [14]:
import os
import shutil
import random

# Set a fixed seed for reproducibility
random.seed(42)

# Define source and destination directories
image_dir = "/content/Whole_Data/images"
label_dir = "/content/Whole_Data/labels"
dest_dir = "/content/Original_Annotation"

# Define the splits (e.g., 70% train, 20% val, 10% test)
train_split = 0.7
val_split = 0.2
test_split = 0.1

# Ensure the destination folders exist
sub_dirs = [
    "images/train", "images/val", "images/test",
    "labels/train", "labels/val", "labels/test"
]

for sub_dir in sub_dirs:
    os.makedirs(os.path.join(dest_dir, sub_dir), exist_ok=True)

# Get a list of all image files
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

# Shuffle the images to get a consistent random split
random.shuffle(image_files)

# Calculate the number of files for each split
total_images = len(image_files)
train_count = int(total_images * train_split)
val_count = int(total_images * val_split)
test_count = total_images - train_count - val_count  # Remaining goes to test

# Split the data
train_files = image_files[:train_count]
val_files = image_files[train_count:train_count + val_count]
test_files = image_files[train_count + val_count:]

# Helper function to copy images and labels
def copy_files(file_list, src_img_dir, src_label_dir, dest_img_dir, dest_label_dir):
    for img_file in file_list:
        img_path = os.path.join(src_img_dir, img_file)
        label_path = os.path.join(src_label_dir, os.path.splitext(img_file)[0] + ".txt")

        # Copy image
        if os.path.exists(img_path):
            shutil.copy2(img_path, os.path.join(dest_img_dir, img_file))

        # Copy label
        if os.path.exists(label_path):
            shutil.copy2(label_path, os.path.join(dest_label_dir, os.path.splitext(img_file)[0] + ".txt"))

# Copy images and labels to respective folders
copy_files(train_files, image_dir, label_dir, os.path.join(dest_dir, "images/train"), os.path.join(dest_dir, "labels/train"))
copy_files(val_files, image_dir, label_dir, os.path.join(dest_dir, "images/val"), os.path.join(dest_dir, "labels/val"))
copy_files(test_files, image_dir, label_dir, os.path.join(dest_dir, "images/test"), os.path.join(dest_dir, "labels/test"))

print("✅ Images and annotations copied successfully.")


✅ Images and annotations copied successfully.


In [15]:
import os
import shutil

# Define the source and destination directories
source_base_dir = "/content/Original_Annotation/images"
destination_base_dir = "/content/Unified_Annotation/images"

# Define the subfolders (train, test, val)
subfolders = ["train", "test", "val"]

# Ensure the destination directories exist
for subfolder in subfolders:
    os.makedirs(os.path.join(destination_base_dir, subfolder), exist_ok=True)

# Move images from each subfolder in the source to the corresponding destination subfolder
for subfolder in subfolders:
    source_subfolder = os.path.join(source_base_dir, subfolder)
    destination_subfolder = os.path.join(destination_base_dir, subfolder)

    # Get the list of image files in the current subfolder
    image_files = [f for f in os.listdir(source_subfolder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    # Move each image file to the destination subfolder
    for img_file in image_files:
        source_path = os.path.join(source_subfolder, img_file)
        destination_path = os.path.join(destination_subfolder, img_file)

        # Move the file
        shutil.move(source_path, destination_path)
        print(f"Moved {img_file} from {source_subfolder} to {destination_subfolder}")

print("✅ All images have been successfully moved.")


Streaming output truncated to the last 5000 lines.
Moved 01136.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation/images/train
Moved 02709.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation/images/train
Moved 03832.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation/images/train
Moved 04888.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation/images/train
Moved 03991.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation/images/train
Moved 01758.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation/images/train
Moved 00960.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation/images/train
Moved 03766.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation/images/train
Moved 03427.jpg from /content/Original_Annotation/images/train to /content/Unified_Annotation

In [16]:
import os

# Define the transformation table based on the new mapping

transform_table = {
    0: 0,      # Pedestrian -> People
    1: 0,      # People -> People
    2: None,   # Bicycle -> Delete
    3: 1,      # Car -> Car
    4: 2,      # Van -> Van
    5: 3,      # Truck -> Truck
    6: None,   # Tricycle -> Delete
    7: None,   # Awning-tricycle -> Delete
    8: 2,      # Bus -> Van
    9: None    # Motor -> Delete
}


# Define custom dataset class names
custom_class_names = [
    "Person", "Car", "Bus", "Truck", "Train", "Crane", "Building", "Plane", "Field", "Bridge", "Roundabout"
]

# Function to unify VisDrone annotations based on the transformation table
def unify_visdrone_labels(annotations_dir, unified_annotations_dir):
    """
    Unifies VisDrone annotations by transforming class labels according to the given mapping.

    Args:
        annotations_dir (str): Path to the directory containing VisDrone annotation files (.txt).
        unified_annotations_dir (str): Path to the directory to save the transformed annotations.
    """
    os.makedirs(unified_annotations_dir, exist_ok=True)

    # Get all annotation files
    annotation_files = [f for f in os.listdir(annotations_dir) if f.endswith('.txt')]

    for file in annotation_files:
        input_file = os.path.join(annotations_dir, file)
        output_file = os.path.join(unified_annotations_dir, file)

        with open(input_file, 'r') as f:
            lines = f.readlines()

        unified_annotations = []
        for line in lines:
            elements = line.strip().split()
            if len(elements) < 5:
                continue  # Skip invalid lines

            # Get original class label
            original_class_id = int(elements[0])

            # Apply transformation
            new_class_id = transform_table.get(original_class_id, None)

            if new_class_id is not None:  # If it's not None, it means it's a valid class
                # Create the unified annotation line
                unified_annotations.append(f"{new_class_id} " + " ".join(elements[1:]))

        # Save the unified annotation file
        if unified_annotations:
            with open(output_file, 'w') as f:
                f.write("\n".join(unified_annotations))

    print(f"✅ Unification completed. Transformed {len(annotation_files)} files.")


In [17]:
#using the Function
# Example usage
annotations_dir = "/content/Original_Annotation/labels/test"  # The directory where VisDrone annotations are stored
unified_annotations_dir = "/content/Unified_Annotation/labels/test"  # The directory to store transformed annotations
unify_visdrone_labels(annotations_dir, unified_annotations_dir)

annotations_dir = "/content/Original_Annotation/labels/train"  # The directory where VisDrone annotations are stored
unified_annotations_dir = "/content/Unified_Annotation/labels/train"  # The directory to store transformed annotations
unify_visdrone_labels(annotations_dir, unified_annotations_dir)

annotations_dir = "/content/Original_Annotation/labels/val"  # The directory where VisDrone annotations are stored
unified_annotations_dir = "/content/Unified_Annotation/labels/val"  # The directory to store transformed annotations
unify_visdrone_labels(annotations_dir, unified_annotations_dir)


✅ Unification completed. Transformed 864 files.
✅ Unification completed. Transformed 6040 files.
✅ Unification completed. Transformed 1725 files.


In [18]:
import shutil

# Define the folder to zip and the output zip file path
folder_to_zip = "/content/Unified_Annotation/labels"
output_zip = "/content/Unified_Annotation.zip"

# Create the zip file
shutil.make_archive(output_zip.replace(".zip", ""), 'zip', folder_to_zip)

# Provide a download link
from google.colab import files
files.download(output_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import shutil
import os

# Authenticate
auth.authenticate_user()

# Zip the folder
folder_path = "/content/Unified_Annotation"
zip_path = "/content/Unified_Annotation.zip"
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)

# Connect to Google Drive
drive_service = build('drive', 'v3')

# Upload the zipped file to Google Drive
file_metadata = {
    'name': 'Unified_Annotation.zip',
    'mimeType': 'application/zip'
}

media = MediaFileUpload(zip_path, mimetype='application/zip')

uploaded_file = drive_service.files().create(body=file_metadata, media_body=media).execute()

print(f"Uploaded to Google Drive: {uploaded_file.get('id')}")


Uploaded to Google Drive: 14WnS_fXZD1rkd0V-qY1_eDKPIkuByblv


In [19]:
# Define the YAML content
yaml_content = """
names:
  - ignored_regions
  - pedestrian
  - people
  - bicycle
  - car
  - van
  - truck
  - tricycle
  - awning-tricycle
  - bus
  - motor
  - others

nc: 12

roboflow:
  license: MIT
  project: visdrone-original
  url: https://universe.roboflow.com/cg-0fmsf/visdrone-original/dataset/1
  version: 1
  workspace: cg-0fmsf

test: /content/Original_Annotation/images/test
train: /content/Original_Annotation/images/train
val: /content/Original_Annotation/images/val
"""

# Define the file path for storing the YAML file
yaml_file_path = '/content/Original_Annotation/VisDrone.yaml'

# Write the content to the YAML file
with open(yaml_file_path, 'w') as file:
    file.write(yaml_content)

print(f"✅ YAML file has been saved at: {yaml_file_path}")


✅ YAML file has been saved at: /content/Original_Annotation/VisDrone.yaml


In [20]:
# Define the YAML content for Unified_Annotation
yaml_content_unified = """
names:
  - people
  - car
  - Bus
  - Truck

nc: 4


test: /content/Unified_Annotation/images/test
train: /content/Unified_Annotation/images/train
val: /content/Unified_Annotation/images/val
"""

# Define the file path for storing the Unified YAML file
yaml_file_path_unified = '/content/Unified_Annotation/VisDrone.yaml'

# Write the content to the YAML file
with open(yaml_file_path_unified, 'w') as file:
    file.write(yaml_content_unified)

print(f"✅ Unified YAML file has been saved at: {yaml_file_path_unified}")


✅ Unified YAML file has been saved at: /content/Unified_Annotation/VisDrone.yaml


In [22]:
#Delete Things we don't need
folders_to_delete = [
    '/content/visdrone-dataset.zip',
    '/content/visdrone_dataset',
    '/content/sample_data',
    '/content/Whole_Data'
]

# Deleting the specified folders or files
for folder in folders_to_delete:
    if os.path.exists(folder):
        if os.path.isdir(folder):
            shutil.rmtree(folder)
            print(f"✅ Deleted folder: {folder}")
        else:
            os.remove(folder)
            print(f"✅ Deleted file: {folder}")
    else:
        print(f"⚠️ Path not found: {folder}")

✅ Deleted file: /content/visdrone-dataset.zip
✅ Deleted folder: /content/visdrone_dataset
✅ Deleted folder: /content/sample_data
✅ Deleted folder: /content/Whole_Data


In [23]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os
import shutil
from google.colab import drive


# Define paths
unified_annotation_dir = '/content/Unified_Annotation'
zip_file_path = '/content/Unified_Annotation.zip'
drive_save_path = '/content/drive/My Drive/Unified_Annotation_Saved.zip'  # Change the filename if needed

# Create a Zip file for Unified_Annotation
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(unified_annotation_dir):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), '/content'))
    print(f"✅ {zip_file_path} has been created.")

# Move the zip file to Google Drive
shutil.move(zip_file_path, drive_save_path)
print(f"✅ File saved to Google Drive: {drive_save_path}")


✅ /content/Unified_Annotation.zip has been created.
✅ File saved to Google Drive: /content/drive/My Drive/Unified_Annotation_Saved.zip


In [ ]:
import os
import shutil

# Path to the zip file
zip_file_path = '/content/VisDrone.zip'

# Path to the destination folder in Google Drive
drive_folder_path = '/content/drive/My Drive/VisDrone/'

# Ensure the folder exists in Google Drive
os.makedirs(drive_folder_path, exist_ok=True)

# Path to where the zip file will be moved
drive_path = os.path.join(drive_folder_path, 'VisDrone.zip')

# Move the zip file to Google Drive
shutil.move(zip_file_path, drive_path)

print(f"✅ File moved to Google Drive at: {drive_path}")


✅ File moved to Google Drive at: /content/drive/My Drive/VisDrone/VisDrone.zip


In [24]:
# Install ultralytics
!pip install ultralytics

# Import the necessary module from ultralytics
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [26]:
import shutil
import os

# Define source and destination directories
folders = ["train", "test", "val"]
source_base = "/content/Unified_Annotation/images"
dest_base = "/content/Original_Annotation/images"

# Ensure destination directories exist
for folder in folders:
    src = os.path.join(source_base, folder)
    dst = os.path.join(dest_base, folder)
    os.makedirs(dst, exist_ok=True)  # Create the destination folder if it doesn't exist

    # Copy files
    for filename in os.listdir(src):
        src_file = os.path.join(src, filename)
        dst_file = os.path.join(dst, filename)
        if os.path.isfile(src_file):  # Ensure it's a file
            shutil.copy2(src_file, dst_file)  # Copy file with metadata

print("Images copied successfully!")


Images copied successfully!


In [ ]:
#Original Model....
#Original Model....
#Original Model....
#Original Model....
#Original Model....
#Original Model....
#Original Model....
#Original Model....
#Original Model....
#Original Model....
#Original Model....
#Original Model....

In [ ]:
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....
#Start Training.....

In [ ]:
#Start Original Nano

from google.colab import drive
from ultralytics import YOLO

# Mount Google Drive
drive.mount('/content/drive')

# Define the YAML file and dataset paths
yaml_file = '/content/Original_Annotation/VisDrone.yaml'  # Path to your YAML file

# Change this to the path where you want to save the results on Google Drive
save_dir = '/content/drive/MyDrive/VisDrone_Original'  # Directory in your Google Drive to save the trained models

# Train the YOLOv8 model (nano version)
model = YOLO('yolov8n.pt')  # YOLOv8 model

# Start training
model.train(
    data=yaml_file,  # Path to the YAML file
    epochs=20,       # Set the number of epochs
    batch=25,        # Adjust batch size as needed
    imgsz=640,       # Image size for training
    project=save_dir,  # Directory to save training results on Google Drive
    name='VisDrone_Original_Nano',  # Name of the experiment
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Original_Annotation/VisDrone.yaml, epochs=20, time=None, patience=100, batch=25, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/VisDrone_Original, name=VisDrone_Original2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, 

train: Scanning /content/Original_Annotation/labels/train... 6040 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6040/6040 [00:12<00:00, 484.77it/s]

train: WARNING ⚠️ /content/Original_Annotation/images/train/00412.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/Original_Annotation/images/train/03491.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/Original_Annotation/images/train/07465.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/Original_Annotation/images/train/08012.jpg: 1 duplicate labels removed


train: New cache created: /content/Original_Annotation/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/Original_Annotation/labels/val... 1725 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1725/1725 [00:02<00:00, 649.50it/s]

val: WARNING ⚠️ /content/Original_Annotation/images/val/00409.jpg: 1 duplicate labels removed


val: New cache created: /content/Original_Annotation/labels/val.cache
Plotting labels to /content/drive/MyDrive/VisDrone_Original/VisDrone_Original2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000625, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005859375000000001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /content/drive/MyDrive/VisDrone_Original/VisDrone_Original2
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/242 [00:00<?, ?it/s]

In [ ]:
#Start Original Small

from google.colab import drive
from ultralytics import YOLO

# Mount Google Drive
drive.mount('/content/drive')

# Define the YAML file and dataset paths
yaml_file = '/content/Original_Annotation/VisDrone.yaml'  # Path to your YAML file

# Change this to the path where you want to save the results on Google Drive
save_dir = '/content/drive/MyDrive/VisDrone_Original'  # Directory in your Google Drive to save the trained models

# Train the YOLOv8 model (nano version)
model = YOLO('yolov8s.pt')  # YOLOv8 model

# Start training
model.train(
    data=yaml_file,  # Path to the YAML file
    epochs=20,       # Set the number of epochs
    batch=25,        # Adjust batch size as needed
    imgsz=640,       # Image size for training
    project=save_dir,  # Directory to save training results on Google Drive
    name='VisDrone_Original_Small',  # Name of the experiment
)

In [ ]:
#Start Original Medium

from google.colab import drive
from ultralytics import YOLO

# Mount Google Drive
drive.mount('/content/drive')

# Define the YAML file and dataset paths
yaml_file = '/content/Original_Annotation/VisDrone.yaml'  # Path to your YAML file

# Change this to the path where you want to save the results on Google Drive
save_dir = '/content/drive/MyDrive/VisDrone_Original'  # Directory in your Google Drive to save the trained models

# Train the YOLOv8 model (nano version)
model = YOLO('yolov8m.pt')  # YOLOv8 model

# Start training
model.train(
    data=yaml_file,  # Path to the YAML file
    epochs=20,       # Set the number of epochs
    batch=25,        # Adjust batch size as needed
    imgsz=640,       # Image size for training
    project=save_dir,  # Directory to save training results on Google Drive
    name='VisDrone_Original_Medium',  # Name of the experiment
)

100%|██████████| 21.5M/21.5M [00:00<00:00, 227MB/s]


Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/Unified_Annotation/VisDrone.yaml, epochs=50, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=/content/Unified_Annotation, name=VisDrone_Unified, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fal

100%|██████████| 755k/755k [00:00<00:00, 42.9MB/s]


Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 167MB/s]


...

       6/50      7.37G      1.436     0.9811     0.9322        369        640: 100%|██████████| 504/504 [03:17<00:00,  2.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:34<00:00,  2.11it/s]


                   all       1725      73033      0.388      0.285      0.268      0.145

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/50      9.48G      1.415     0.9643     0.9278        288        640: 100%|██████████| 504/504 [03:07<00:00,  2.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:30<00:00,  2.35it/s]


                   all       1725      73033      0.404       0.29      0.274      0.147

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       8/50      5.65G      1.405     0.9472     0.9224        974        640:  72%|███████▏  | 364/504 [02:19<01:11,  1.97it/s]